In [8]:
import pandas as pd
import textwrap
import builtins
import time


In [6]:
import gspread

In [2]:
from google.oauth2.service_account import Credentials
import json

In [3]:
pip install openai

     -------------------------------------- 65.6/65.6 kB 878.7 kB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 225.4/225.4 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 85.5/85.5 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 75.9/75.9 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 76.9/76.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 381.9/381.9 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 1.9/1.9 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.0.1
    Uninstalling typing_extensions-4.0.1:
      Successfully uninstalled typing_extensions-4.0.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
import openai

In [ ]:
def wprint(*args, width=70, **kwargs):
    wrapper = textwrap.TextWrapper(width=width)
    wrapped_args = [wrapper.fill(str(arg)) for arg in args]
    builtins.print(*wrapped_args, **kwargs)


def get_completion(message, agent, funcs, thread, client):
    # Create new message in the thread
    message_response = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message
    )


    # Run the thread
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=agent.id,
    )


    while True:
        # Wait until run completes
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )


        if run.status in ['queued', 'in_progress']:
            time.sleep(1)
            continue


        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []
            for tool_call in tool_calls:
                print(f"Debug: Calling function {tool_call.function.name}", flush=True)


                wprint(f'\033[31mFunction: {tool_call.function.name}\033[0m')
                func = next((f for f in funcs if f.__name__ == tool_call.function.name), None)
                if func:
                    try:
                        # Assuming arguments are parsed correctly
                        func_instance = func(**eval(tool_call.function.arguments))  # Consider safer alternatives to eval
                        output = func_instance.run()


                        # Ensure output is a string
                        if not isinstance(output, str):
                            output = str(output)
                    except Exception as e:
                        output = f"Error: {e}"
                else:
                    output = "Function not found"
                wprint(f"\033[33m{tool_call.function.name}: {output}\033[0m")
                tool_outputs.append({"tool_call_id": tool_call.id, "output": output})


            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
        elif run.status == "failed":
            raise Exception(f"Run Failed. Error: {run.last_error}")
        else:
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            )
            latest_message = messages.data[0].content[0].text.value
            return latest_message


In [ ]:
class DataCleanerAssistant :
    openai_schema = {
        "name": "data_cleaner_and_uploader",
        "description": "Cleans a dataframe and uploads it to a specified Google Sheet",
        "parameters": {
            "type": "object",
            "properties": {
                "dataframe_path": {"type": "string", "description": "Path to the dataframe file"},
                "sheet_name": {"type": "string", "description": "Name of the Google Sheet to upload data"}
            },
            "required": ["dataframe_path", "sheet_name"]
        }
    }

    def __init__(self, dataframe_path, sheet_name) -> None:
        self.dataframe_path = dataframe_path
        self.sheet_name = sheet_name

    def clean_dataframe(self, df):
        df.dropna(inplace=True)

        df = df.convert_dtypes()

        df.drop_duplicates(inplace=True)

        return df
    
    def upload_sheet(self, df):
        creds = Credentials.from_service_account_file()
        client = gspread.authorize(creds)
        sheet = client.open(self.sheet_name).sheet1
        sheet.clear
        sheet.update([df.columns.values.tolist()] + df.values.tolist())
    def run(self):
        try:
            df = pd.read_csv(self.dataframe_path)
            clean_df = self.clean_dataframe(df)
            self.upload_to_sheet(clean_df)
            return "Data cleaned and uploaded successfully."
        except Exception as e:
            return f"Error: {str(e)}"
        



In [ ]:
data_cleaner_and_uploader_tool = DataCleanerAssistant.openai_schema

assistant_tools = [data_cleaner_and_uploader_tool] 

# Create the assistant with the specified tools
data_assistant_ = client.beta.assistants.create(
    name='Data Assistant',
    instructions="Take in data frames and clean it and transform it into a google sheet.",
    model="gpt-3.5-turbo-1106",
    tools=assistant_tools
)


In [ ]:
# Create a new thread for interaction
thread = client.beta.threads.create()

# Example user input
user_input = "Please clean the data in 'data.csv' and upload it to 'MySheet'"
message = get_completion(user_input, data_assistant, [DataCleanerAssistant], thread, client)
print(message)
